<a href="https://colab.research.google.com/github/iqrasarwar/ComputerVision/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**WithOut Transfer Learning**



In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
import tensorflow
import keras

In [2]:
images = []
labels = []

In [5]:
dataset_path = '/content/drive/MyDrive/ComputerVisionCoLabFiles/CNN/train/'
folders = os.listdir(dataset_path)
folders

['sad', 'angry', 'neutral', 'disgust', 'fear', 'surprise', 'happy']

In [6]:
name_mapping_dict = {'angry':0, 'disgust':1, 'fear':2, 'happy':3, 'neutral':4, 'sad':5, 'surprise':6}
inv_name_mapping_dict =  {v: k for k, v in name_mapping_dict.items()}
print(inv_name_mapping_dict)

{0: 'angry', 1: 'disgust', 2: 'fear', 3: 'happy', 4: 'neutral', 5: 'sad', 6: 'surprise'}


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
img_size = 100

In [9]:
for folder in folders:
  print(f'Processing folder: {folder}')
  filenames = os.listdir(dataset_path+folder+'/')
  for fname in filenames:
    img = plt.imread(dataset_path+folder+'/'+fname)
    img = cv2.resize(img, (img_size, img_size))
    images.append(img)
    labels.append(name_mapping_dict[folder])

Processing folder: sad
Processing folder: angry
Processing folder: neutral
Processing folder: disgust
Processing folder: fear
Processing folder: surprise
Processing folder: happy


In [11]:
images = np.expand_dims(images, axis=3)
images = np.array(images)
labels = np.array(labels)

In [12]:
print('images shape : ', images.shape)
print('labels shape : ', labels.shape)

images shape :  (28847, 100, 100, 1)
labels shape :  (28847,)


In [13]:
classes = np.unique(labels)
classes_num = len(classes)
print('Total number of outputs : ', classes_num)
print('Output classes : ', classes)

Total number of outputs :  7
Output classes :  [0 1 2 3 4 5 6]


In [14]:
images = np.array(images) / 255

In [16]:
labels_one_hot = to_categorical(labels)

In [17]:
labels[303]

5

In [18]:
labels_one_hot[303]

array([0., 0., 0., 0., 0., 1., 0.], dtype=float32)

In [19]:
# Shuffling and Splittng Dataset
from sklearn.model_selection import train_test_split

train_images, test_images, train_labels, test_labels = train_test_split(images, labels_one_hot, test_size=0.10, shuffle=True, random_state=0)


print('Training data shape : ', train_images.shape, train_labels.shape)
print('Testing data shape : ', test_images.shape, test_labels.shape)

Training data shape :  (25962, 100, 100, 1) (25962, 7)
Testing data shape :  (2885, 100, 100, 1) (2885, 7)


In [20]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten 

# Same padding: This padding ensures that the output layer has the same size as the input layer
model = Sequential()
model.add(Conv2D(32, 4, padding="same", activation="relu", input_shape=(img_size, img_size,1)))
model.add(MaxPool2D())

model.add(Conv2D(32, 4, padding="same", activation="relu"))
model.add(MaxPool2D())

model.add(Conv2D(64, 4, padding="same", activation="relu"))
model.add(MaxPool2D())

model.add(Conv2D(64, 4, padding="same", activation="relu"))
model.add(MaxPool2D())

model.add(Flatten())
model.add(Dense(128,activation="relu"))
model.add(Dense(classes_num, activation="softmax"))

In [21]:
model.compile(optimizer='adam', loss= 'categorical_crossentropy', metrics=['accuracy'])

I ran below cell model.fit for first time it started from 0.3 accuracy and improved gradually. I accediently again ran it, it lost the previous result(output of epochs) and started epochs from 1 but this time from accuracy of 0.97. It took almost 3 hours 45 mints for first run. I can now re run from start to show the results but I don't know how much time it will take in this second run I don't have the enough time to run it for the 3rd time:( So leaving it as it is. 

In [23]:
history = model.fit(train_images, train_labels, batch_size=128, epochs=25, verbose=1, 
                   validation_data=(test_images, test_labels))

Epoch 1/25
203/203 [==============================] - 534s 3s/step - loss: 0.0747 - accuracy: 0.9765 - val_loss: 4.0605 - val_accuracy: 0.5567
Epoch 2/25
203/203 [==============================] - 531s 3s/step - loss: 0.0693 - accuracy: 0.9784 - val_loss: 4.3510 - val_accuracy: 0.5504
Epoch 3/25
203/203 [==============================] - 528s 3s/step - loss: 0.0701 - accuracy: 0.9790 - val_loss: 4.1418 - val_accuracy: 0.5560
Epoch 4/25
203/203 [==============================] - 527s 3s/step - loss: 0.0727 - accuracy: 0.9778 - val_loss: 4.1006 - val_accuracy: 0.5473
Epoch 5/25
203/203 [==============================] - 525s 3s/step - loss: 0.0605 - accuracy: 0.9809 - val_loss: 4.3318 - val_accuracy: 0.5511
Epoch 6/25
203/203 [==============================] - 527s 3s/step - loss: 0.0525 - accuracy: 0.9842 - val_loss: 4.6167 - val_accuracy: 0.5445
Epoch 7/25
203/203 [==============================] - 525s 3s/step - loss: 0.0849 - accuracy: 0.9739 - val_loss: 4.2712 - val_accuracy: 0.5421

In [24]:
[test_loss, test_acc] = model.evaluate(test_images, test_labels)
print("Evaluation result on Test Data : Loss = {}, accuracy = {}".format(test_loss, test_acc*100))

91/91 [==============================] - 13s 147ms/step - loss: 4.7752 - accuracy: 0.5490
Evaluation result on Test Data : Loss = 4.775172710418701, accuracy = 54.90468144416809


In [29]:
model.save(dataset_path+'model.h5')

**Using Transfer Learning**

In [25]:
seed_value= 0

import os
os.environ['PYTHONHASHSEED']=str(seed_value)

import random
random.seed(seed_value)

import cv2

import numpy as np
np.random.seed(seed_value)

import tensorflow as tf
tf.random.set_seed(seed_value)

import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical

In [26]:
images1 = []
labels1 = []

In [27]:
folders = os.listdir(dataset_path)
folders

['sad', 'angry', 'neutral', 'disgust', 'fear', 'surprise', 'happy']

In [28]:
img_size = 120

In [32]:
name_mapping_dict = {'angry':0, 'disgust':1, 'fear':2, 'happy':3, 'neutral':4, 'sad':5, 'surprise':6}
inv_name_mapping_dict =  {v: k for k, v in name_mapping_dict.items()}
print(inv_name_mapping_dict)

{0: 'angry', 1: 'disgust', 2: 'fear', 3: 'happy', 4: 'neutral', 5: 'sad', 6: 'surprise'}


In [33]:
img_size = 225

In [34]:
for folder in folders:
  print(f'Processing folder: {folder}')
  filenames = os.listdir(dataset_path+folder+'/')
  for fname in filenames:
    img = plt.imread(dataset_path+folder+'/'+fname)
    img = cv2.resize(img, (img_size, img_size))
    images1.append(img)
    labels1.append(name_mapping_dict[folder])

Processing folder: sad
Processing folder: angry
Processing folder: neutral
Processing folder: disgust
Processing folder: fear
Processing folder: surprise
Processing folder: happy


In [35]:
images1 = np.expand_dims(images1, axis=3)
images1 = np.array(images1)
labels1 = np.array(labels1)

In [36]:
print('images shape : ', images1.shape)
print('labels shape : ', labels1.shape)

images shape :  (28847, 225, 225, 1)
labels shape :  (28847,)


In [37]:
classes = np.unique(labels)
classes_num = len(classes)
print('Total number of outputs : ', classes_num)
print('Output classes : ', classes)

Total number of outputs :  7
Output classes :  [0 1 2 3 4 5 6]


In [38]:
images = np.array(images) / 255

In [39]:
labels_one_hot = to_categorical(labels)

In [40]:
# Shuffling and Splittng Dataset
from sklearn.model_selection import train_test_split

train_images, test_images, train_labels, test_labels = train_test_split(images, labels_one_hot, test_size=0.20, shuffle=True, random_state=0)


print('Training data shape : ', train_images.shape, train_labels.shape)
print('Testing data shape : ', test_images.shape, test_labels.shape)

Training data shape :  (23077, 100, 100, 1) (23077, 7)
Testing data shape :  (5770, 100, 100, 1) (5770, 7)
